# Setup

In [16]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import shutil
from pathlib import Path
import re
import uuid
from tensorflow.keras.metrics import Precision, Recall
import shutil
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.callbacks import *
import time
from sklearn.metrics import accuracy_score
import gc

In [17]:
set_global_policy('mixed_float16')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.enable_tensor_float_32()
        tf.config.set_soft_device_placement(True)
        tf.debugging.set_log_device_placement(False)
    except RuntimeError as e:
        print(e)

In [18]:
tf.config.threading.set_intra_op_parallelism_threads(0)
tf.config.threading.set_inter_op_parallelism_threads(0)

In [19]:
TRAIN = os.path.join('data', 'training')
TEST = os.path.join('data', 'test')
ARCH = os.path.join('data', 'archive')

In [20]:
class VerificationAccuracyEarlyStopping(Callback):
    """Custom early stopping based on verification accuracy degradation"""
    
    def __init__(self, validation_data=None, patience=5, min_delta=0.01, 
                 min_verification_acc=0.90, restore_best_weights=True, verbose=1):
        super().__init__()
        self.validation_data = validation_data
        self.patience = patience
        self.min_delta = min_delta
        self.min_verification_acc = min_verification_acc
        self.restore_best_weights = restore_best_weights
        self.verbose = verbose
        self.best_verification_acc = 0
        self.best_weights = None
        self.wait = 0
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is not None:
            val_acc = self.calculate_verification_accuracy()
            
            if self.verbose:
                print(f"Epoch {epoch + 1} - Verification Accuracy: {val_acc:.4f}")
            
            if val_acc > self.best_verification_acc + self.min_delta:
                self.best_verification_acc = val_acc
                self.wait = 0
                if self.restore_best_weights:
                    self.best_weights = self.model.get_weights()
            else:
                self.wait += 1
                
            if val_acc < self.min_verification_acc:
                print(f"Verification accuracy dropped below {self.min_verification_acc}. Stopping training.")
                self.model.stop_training = True
                
            if self.wait >= self.patience:
                print(f"Verification accuracy hasn't improved for {self.patience} epochs. Stopping training.")
                self.model.stop_training = True
                
            if self.model.stop_training and self.restore_best_weights and self.best_weights:
                self.model.set_weights(self.best_weights)
                print(f"Restored best weights with verification accuracy: {self.best_verification_acc:.4f}")
    
    def calculate_verification_accuracy(self):
        """Calculate verification accuracy on validation pairs"""
        anchor_imgs, comparison_imgs, labels = self.validation_data
        
        batch_size = 32
        predictions = []
        
        for i in range(0, len(anchor_imgs), batch_size):
            try:
                batch_anchors = anchor_imgs[i:i+batch_size]
                batch_comparisons = comparison_imgs[i:i+batch_size]
                
                processed_anchors = []
                processed_comparisons = []
                
                for img in batch_anchors:
                    if isinstance(img, str):
                        processed_img = preprocess_with_augmentation(img, is_training=False)
                    else:
                        processed_img = tf.cast(img, tf.float32) / 255.0 if tf.reduce_max(img) > 1.0 else img
                    processed_anchors.append(processed_img)
                
                for img in batch_comparisons:
                    if isinstance(img, str):
                        processed_img = preprocess_with_augmentation(img, is_training=False)
                    else:
                        processed_img = tf.cast(img, tf.float32) / 255.0 if tf.reduce_max(img) > 1.0 else img
                    processed_comparisons.append(processed_img)
                
                processed_anchors = tf.stack(processed_anchors)
                processed_comparisons = tf.stack(processed_comparisons)
                
                # Handle both single and multi-output models
                model_output = self.model([processed_anchors, processed_comparisons])
                if isinstance(model_output, list):
                    batch_preds = model_output[-1]  # Use the prediction output
                else:
                    batch_preds = model_output
                    
                predictions.extend(batch_preds.numpy().flatten())
            except Exception as e:
                print(f"Error in batch processing: {e}")
                continue
        
        if len(predictions) == 0:
            return 0.0
            
        binary_preds = (np.array(predictions) > 0.5).astype(int)
        return accuracy_score(labels[:len(binary_preds)], binary_preds)


In [21]:
class GPUUtilizationMonitor(Callback):
    """Monitor and optimize GPU utilization during training"""
    
    def __init__(self, target_utilization=0.95):
        super().__init__()
        self.target_utilization = target_utilization
        
    def on_epoch_begin(self, epoch, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()

# Datamanipulation

In [22]:
def create_optimized_pairs_from_directory(directory, max_people=1500, max_pairs_per_person=200):
    person_dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]

    if len(person_dirs) > max_people:
        person_dirs = np.random.choice(person_dirs, max_people, replace=False)
    
    print(f"Using {len(person_dirs)} people (limited from potentially more)")
    pos_anchor_paths = []
    pos_comparison_paths = []
    neg_anchor_paths = []
    neg_comparison_paths = []

    for idx, person in enumerate(person_dirs):
        person_path = os.path.join(directory, person)
        person_images = [os.path.join(person_path, f) for f in os.listdir(person_path) 
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        if len(person_images) < 2:
            continue

        pair_count = 0
        for i in range(len(person_images)):
            if pair_count >= max_pairs_per_person:
                break
            for j in range(i+1, len(person_images)):
                if pair_count >= max_pairs_per_person:
                    break
                pos_anchor_paths.append(person_images[i])
                pos_comparison_paths.append(person_images[j])
                pair_count += 1

    print(f"Created {len(pos_anchor_paths)} positive pairs")

    target_negative_pairs = len(pos_anchor_paths)
    
    for idx, person in enumerate(person_dirs):
        if len(neg_anchor_paths) >= target_negative_pairs:
            break
            
        person_path = os.path.join(directory, person)
        person_images = [os.path.join(person_path, f) for f in os.listdir(person_path) 
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        if len(person_images) < 1:
            continue

        other_people = [p for p in person_dirs if p != person]
        if not other_people:
            continue

        for anchor_img in person_images: 
            if len(neg_anchor_paths) >= target_negative_pairs:
                break

            sampled_others = np.random.choice(other_people, min(len(other_people), 20), replace=False)
            
            for other_person in sampled_others:
                if len(neg_anchor_paths) >= target_negative_pairs:
                    break
                    
                other_person_path = os.path.join(directory, other_person)
                other_person_images = [os.path.join(other_person_path, f) 
                                       for f in os.listdir(other_person_path) 
                                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

                if not other_person_images:
                    continue

                for _ in range(min(8, len(other_person_images))):  
                    if len(neg_anchor_paths) >= target_negative_pairs:
                        break
                    negative_img = np.random.choice(other_person_images)
                    neg_anchor_paths.append(anchor_img)
                    neg_comparison_paths.append(negative_img)

    print(f"Created {len(neg_anchor_paths)} negative pairs")
    print(f"✅ Final dataset: {len(pos_anchor_paths)} positive, {len(neg_anchor_paths)} negative pairs")
    print(f"✅ Total pairs: {len(pos_anchor_paths) + len(neg_anchor_paths)}")
    
    all_anchor_paths = pos_anchor_paths + neg_anchor_paths
    all_comparison_paths = pos_comparison_paths + neg_comparison_paths

    positive_labels = tf.ones(len(pos_anchor_paths))
    negative_labels = tf.zeros(len(neg_anchor_paths))
    all_labels = tf.concat([positive_labels, negative_labels], axis=0)

    return all_anchor_paths, all_comparison_paths, all_labels

In [23]:
def advanced_augmentation():
    return tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomContrast(0.1),
        tf.keras.layers.RandomBrightness(0.1),
    ])


In [24]:
GLOBAL_AUGMENTATION = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])

In [25]:
def preprocess_with_augmentation(img_path, is_training=True):
    byte_img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(byte_img, channels=3)
    img = tf.image.resize(img, (100, 100))  # Larger input size
    
    img = tf.cast(img, tf.float32) / 255.0
    
    if is_training:
        img = GLOBAL_AUGMENTATION(img)
    
    return img

In [26]:
def create_hard_negative_pairs(embedding_model, directory, num_hard_negatives=1000):
    """Create pairs where the model currently struggles (hard negatives)"""
    person_dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    
    all_embeddings = {}
    all_image_paths = {}
    
    for person in person_dirs[:100]: 
        person_path = os.path.join(directory, person)
        images = [os.path.join(person_path, f) for f in os.listdir(person_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))][:10]
        
        if len(images) < 2:
            continue
            
        embeddings = []
        for img_path in images:
            img = preprocess_with_augmentation(img_path, is_training=False)
            img = tf.expand_dims(img, 0)
            emb = embedding_model(img)
            embeddings.append(emb.numpy()[0])
        
        all_embeddings[person] = np.array(embeddings)
        all_image_paths[person] = images

    hard_anchor_paths = []
    hard_negative_paths = []
    
    people_list = list(all_embeddings.keys())
    
    for i, person1 in enumerate(people_list):
        for j, person2 in enumerate(people_list[i+1:], i+1):
            # Calculate all pairwise distances between person1 and person2
            for k, emb1 in enumerate(all_embeddings[person1]):
                for l, emb2 in enumerate(all_embeddings[person2]):
                    distance = np.linalg.norm(emb1 - emb2)
                    
                    if distance < 0.5:  # Threshold for "hard negative"
                        hard_anchor_paths.append(all_image_paths[person1][k])
                        hard_negative_paths.append(all_image_paths[person2][l])
                        
                        if len(hard_anchor_paths) >= num_hard_negatives:
                            return hard_anchor_paths, hard_negative_paths
    
    return hard_anchor_paths, hard_negative_paths

In [27]:
def create_optimized_data_pipeline(anchors, comparisons, labels, batch_size=64, 
                                 prefetch_buffer=tf.data.AUTOTUNE, num_parallel_calls=tf.data.AUTOTUNE):
    """Create highly optimized data pipeline for maximum GPU utilization"""
    
    anchor_ds = tf.data.Dataset.from_tensor_slices(anchors)
    comparison_ds = tf.data.Dataset.from_tensor_slices(comparisons)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    dataset = tf.data.Dataset.zip((anchor_ds, comparison_ds, labels_ds))
    dataset = dataset.shuffle(buffer_size=min(10000, len(anchors)))
    dataset = dataset.map(
        lambda a, c, l: (
            (preprocess_with_augmentation(a, is_training=True),
             preprocess_with_augmentation(c, is_training=True)),
            tf.cast(l, tf.float32)
        ),
        num_parallel_calls=num_parallel_calls
    )


    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(prefetch_buffer)
    if len(anchors) < 50000:
        dataset = dataset.cache()
    
    return dataset

In [28]:
def create_validation_data(test_directory, num_pairs=1000):
    """Create validation data for verification accuracy monitoring"""
    
    person_dirs = [d for d in os.listdir(test_directory) 
                   if os.path.isdir(os.path.join(test_directory, d))]
    
    anchors, comparisons, labels = [], [], []
    
    for person in person_dirs[:50]:  # Limit for efficiency
        person_path = os.path.join(test_directory, person)
        images = [os.path.join(person_path, f) for f in os.listdir(person_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))][:10]
        
        if len(images) >= 2:
            for i in range(min(5, len(images)-1)):
                anchors.append(images[i])
                comparisons.append(images[i+1])
                labels.append(1)

    for i in range(len(anchors)):
        if len(person_dirs) > 1:
            # Select different person
            other_person = random.choice([p for p in person_dirs[:50] if p != person_dirs[i % len(person_dirs)]])
            other_person_path = os.path.join(test_directory, other_person)
            other_images = [os.path.join(other_person_path, f) for f in os.listdir(other_person_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if other_images:
                anchors.append(anchors[i])  # Same anchor
                comparisons.append(random.choice(other_images))
                labels.append(0)
    
    return anchors[:num_pairs], comparisons[:num_pairs], labels[:num_pairs]


# Help functions

In [29]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, in_embed, valid_embed):
        return tf.math.abs(in_embed - valid_embed)

def create_optimized_embedding():
    inputs = tf.keras.Input(shape=(100, 100, 3), name="input_img")

    x = tf.keras.layers.SeparableConv2D(64, (10, 10), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.SeparableConv2D(128, (7, 7), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.SeparableConv2D(128, (4, 4), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.SeparableConv2D(256, (4, 4), activation='relu', padding='same')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    outputs = tf.keras.layers.Dense(512, activation='sigmoid', dtype='float32')(x) 
    
    return tf.keras.Model(inputs=inputs, outputs=outputs, name='optimized_embedding')

class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin
    
    def call(self, y_true, y_pred):
        # y_pred is the distance between embeddings
        # y_true is 1 for same person, 0 for different
        y_true = tf.cast(y_true, tf.float32)
        
        # For same person pairs, minimize distance
        # For different person pairs, maximize distance up to margin
        loss = y_true * tf.square(y_pred) + \
               (1 - y_true) * tf.square(tf.maximum(0.0, self.margin - y_pred))
        return tf.reduce_mean(loss)

In [30]:
def create_advanced_embedding_with_attention():
    inputs = tf.keras.Input(shape=(100, 100, 3), name="input_img")  # Larger input size
    
    # Feature extraction backbone
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    attention = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)
    x = tf.keras.layers.Multiply()([x, attention])
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    embeddings = tf.keras.layers.Dense(512, activation=None, dtype='float32')(x)
    embeddings = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(embeddings)
    
    return tf.keras.Model(inputs=inputs, outputs=embeddings, name='advanced_embedding')

In [31]:
class TripletLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin
    
    def call(self, y_true, y_pred):
        # y_pred should be [anchor, positive, negative] embeddings
        anchor, positive, negative = tf.split(y_pred, 3, axis=1)
        
        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        
        loss = tf.maximum(0.0, pos_dist - neg_dist + self.margin)
        return tf.reduce_mean(loss)

class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin
    
    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32) 
        loss = y_true * tf.square(y_pred) + \
               (1 - y_true) * tf.square(tf.maximum(0.0, self.margin - y_pred))
        return tf.reduce_mean(loss)


In [32]:
class EuclideanDistance(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def call(self, inputs):
        anchor, comparison = inputs
        return tf.sqrt(tf.reduce_sum(tf.square(anchor - comparison), axis=-1, keepdims=True))


In [33]:
def create_advanced_siamese_model():
    embedding_network = create_advanced_embedding_with_attention()
    anchor_input = Input(shape=(100, 100, 3), name='anchor')
    comparison_input = Input(shape=(100, 100, 3), name='comparison')
    anchor_embedding = embedding_network(anchor_input)
    comparison_embedding = embedding_network(comparison_input)
    distance = EuclideanDistance(name='euclidean_distance')([anchor_embedding, comparison_embedding])
    prediction = Dense(1, activation='sigmoid', name='prediction', dtype='float32')(distance)

    
    model = Model(inputs=[anchor_input, comparison_input], 
                  outputs=[distance, prediction],  
                  name='advanced_siamese')
    
    return model, embedding_network

In [34]:
def create_multi_loss_model():
    siamese_model, embedding_model = create_advanced_siamese_model()
    
    siamese_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss={
            'euclidean_distance': ContrastiveLoss(margin=1.0),
            'dense': 'binary_crossentropy'
        },
        loss_weights={'euclidean_distance': 1.0, 'prediction': 0.5},
        metrics=['accuracy']
    )
    
    return siamese_model, embedding_model

In [35]:
class Cast(Layer):
    def __init__(self, dtype=tf.float32, **kwargs):
        super(Cast, self).__init__(**kwargs)
        self.dtype_to_cast = dtype

    def call(self, inputs):
        return tf.cast(inputs, self.dtype_to_cast)

# Model

In [36]:
def load_and_enhance_existing_model(model_path='face_verification_v2.h5'):
    """Load existing model and prepare for enhanced training"""
    print(f"Loading existing model from {model_path}")
    
    try:
        # Comprehensive custom objects dictionary
        custom_objects = {
            'L1Dist': L1Dist,
            'EuclideanDistance': EuclideanDistance,
            'ContrastiveLoss': ContrastiveLoss,
            'Cast': Cast,  # Add Cast layer
            'BinaryCrossentropy': tf.losses.BinaryCrossentropy,
            'cast': tf.cast,  # Add cast function
            'l2_normalize': tf.nn.l2_normalize,
        }
        
        # Try loading with custom objects
        with tf.keras.utils.custom_object_scope(custom_objects):
            model = load_model(model_path, compile=False)
            
        print("✅ Model loaded successfully!")
        print(f"Model outputs: {[output.name for output in model.outputs]}")
        
        # Recompile the model
        if len(model.outputs) > 1:
            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                loss={
                    'euclidean_distance': ContrastiveLoss(margin=1.0),
                    'prediction': 'binary_crossentropy'
                },
                loss_weights={'euclidean_distance': 1.0, 'prediction': 0.5},
                metrics={'prediction': 'accuracy'},
                run_eagerly=False
            )
        else:
            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss='binary_crossentropy',
                metrics=['accuracy'],
                run_eagerly=False
            )
        
        print("✅ Model recompiled successfully!")
        return model
        
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        print("Creating new model...")
        
        model, _ = create_multi_loss_model()
        print("✅ New model created and compiled successfully!")
        return model


In [37]:
def train_easy_model(model, anchors, comparisons, labels, epochs=10, batch_size=32):
    anchor_ds = tf.data.Dataset.from_tensor_slices(anchors)
    comparison_ds = tf.data.Dataset.from_tensor_slices(comparisons)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((anchor_ds, comparison_ds, labels_ds))
    dataset = dataset.map(lambda a, c, l: (
        preprocess_with_augmentation(a, is_training=True),
        preprocess_with_augmentation(c, is_training=True),
        l
    ), num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    model.fit(
        dataset,
        epochs=epochs,
        callbacks=[
            EarlyStopping(patience=5, restore_best_weights=True),
            ReduceLROnPlateau(factor=0.5, patience=3),
        ]
    )

In [38]:
def curriculum_training(model, train_directory, test_directory, total_epochs=50):
    print("Stage 1: Easy examples (clear differences)")
    easy_anchors, easy_comparisons, easy_labels = create_optimized_pairs_from_directory(
        train_directory, max_people=500, max_pairs_per_person=20
    )
    
    train_easy_model(model, easy_anchors, easy_comparisons, easy_labels, epochs=15)
    
    print("Stage 2: Adding medium difficulty examples")
    medium_anchors, medium_comparisons, medium_labels = create_optimized_pairs_from_directory(
        train_directory, max_people=1000, max_pairs_per_person=30
    )
    
    all_anchors = easy_anchors + medium_anchors
    all_comparisons = easy_comparisons + medium_comparisons
    all_labels = easy_labels + medium_labels
    
    train_easy_model(model, all_anchors, all_comparisons, all_labels, epochs=20)
    
    print("Stage 3: Adding hard negatives")
    embedding_model = model.layers[2] 
    hard_anchors, hard_negatives = create_hard_negative_pairs(embedding_model, train_directory)
    
    all_anchors.extend(hard_anchors)
    all_comparisons.extend(hard_negatives)
    all_labels.extend([0] * len(hard_anchors))
    
    train_easy_model(model, all_anchors, all_comparisons, all_labels, epochs=15)
    
    return model


In [39]:
def train_to_100_percent_accuracy():
    siamese_model, embedding_model = create_multi_loss_model()
    
    trained_model = curriculum_training(
        siamese_model, 
        TRAIN, 
        TEST,
        total_epochs=50
    )
    
    print("Fine-tuning on hard examples...")
    hard_anchors, hard_negatives = create_hard_negative_pairs(
        embedding_model, 
        'data/training'
    )
    

    trained_model.save('face_verification_advanced.h5')
    
    return trained_model

In [40]:
def advanced_curriculum_training(model, train_directory, test_directory, 
                               initial_epochs=20, fine_tune_epochs=30, batch_size=64):
    """Advanced curriculum training with GPU optimization and early stopping"""
    
    print("Creating validation data for monitoring...")
    val_anchors, val_comparisons, val_labels = create_validation_data(test_directory)
    
    print("\n=== Stage 1: Enhanced Training on Diverse Data ===")

    train_anchors, train_comparisons, train_labels = create_optimized_pairs_from_directory(
        train_directory, max_people=2000, max_pairs_per_person=50
    )
    
    print(f"Created {len(train_anchors)} training pairs")

    train_dataset = create_optimized_data_pipeline(
        train_anchors, train_comparisons, train_labels, batch_size=batch_size
    )
    
    callbacks = [
        VerificationAccuracyEarlyStopping(
            validation_data=(val_anchors, val_comparisons, val_labels),
            patience=7,
            min_verification_acc=0.70,
            verbose=1
        ),
        GPUUtilizationMonitor(),
        ReduceLROnPlateau(
            monitor='loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            'face_verification_enhanced.h5',
            monitor='loss',
            save_best_only=True,
            verbose=1
        )
    ]
    
    
    print("Starting enhanced training...")
    history1 = model.fit(
        train_dataset,
        epochs=initial_epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n=== Stage 2: Fine-tuning on Hard Examples ===")
    embedding_model = None
    for layer in model.layers:
        if 'embedding' in layer.name.lower():
            embedding_model = layer
            break
    
    if embedding_model is None:
        embedding_model = Model(inputs=model.input[0], outputs=model.layers[-3].output)
    
    hard_anchors, hard_negatives = create_hard_negative_pairs(
        embedding_model, train_directory, num_hard_negatives=2000
    )

    all_anchors = train_anchors + hard_anchors
    all_comparisons = train_comparisons + hard_negatives
    all_labels = train_labels + [0] * len(hard_anchors)
    
    print(f"Added {len(hard_anchors)} hard negative pairs")

    fine_tune_dataset = create_optimized_data_pipeline(
        all_anchors, all_comparisons, all_labels, batch_size=batch_size//2  
    )
    
    model.optimizer.learning_rate = 5e-6
    callbacks[0].min_verification_acc = 0.85  
    callbacks[0].patience = 5
    
    print("Starting fine-tuning...")
    history2 = model.fit(
        fine_tune_dataset,
        epochs=fine_tune_epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n=== Stage 3: Final Optimization ===")
    
    model.optimizer.learning_rate = 1e-6
    
    final_dataset = create_optimized_data_pipeline(
        all_anchors, all_comparisons, all_labels, batch_size=32
    )

    callbacks[0].min_verification_acc = 0.95 
    callbacks[0].patience = 3
    
    print("Starting final optimization...")
    history3 = model.fit(
        final_dataset,
        epochs=20,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, [history1, history2, history3]

In [41]:
def evaluate_final_performance(model, test_directory):   
    print("\n=== Final Performance Evaluation ===")
    
    test_anchors, test_comparisons, test_labels = create_validation_data(test_directory, num_pairs=2000)
    predictions = []
    batch_size = 32
    
    for i in range(0, len(test_anchors), batch_size):
        batch_anchors = test_anchors[i:i+batch_size]
        batch_comparisons = test_comparisons[i:i+batch_size]
        
        processed_anchors = tf.stack([preprocess_with_augmentation(img, is_training=False) 
                                    for img in batch_anchors])
        processed_comparisons = tf.stack([preprocess_with_augmentation(img, is_training=False) 
                                        for img in batch_comparisons])
        
        _, batch_preds = model([processed_anchors, processed_comparisons])
        predictions.extend(batch_preds.numpy().flatten())
    
    binary_preds = (np.array(predictions) > 0.5).astype(int)
    final_accuracy = accuracy_score(test_labels, binary_preds)
    
    print(f"Final Verification Accuracy: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")
    
    from sklearn.metrics import precision_score, recall_score, f1_score
    precision = precision_score(test_labels, binary_preds)
    recall = recall_score(test_labels, binary_preds)
    f1 = f1_score(test_labels, binary_preds)
    
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return final_accuracy, precision, recall, f1

In [42]:
def train_to_perfect_verification():
    
    print("Loading existing model and enhancing it...")
    model = load_and_enhance_existing_model('face_verification_v2.h5')
    
    print("Starting advanced curriculum training...")
    enhanced_model, training_histories = advanced_curriculum_training(
        model, TRAIN, TEST, 
        initial_epochs=25, 
        fine_tune_epochs=35, 
        batch_size=64
    )
    
    print("Evaluating final performance...")
    final_accuracy, precision, recall, f1 = evaluate_final_performance(enhanced_model, TEST)

    enhanced_model.save('face_verification_perfect.h5')
    print("Enhanced model saved as 'face_verification_perfect.h5'")
    
    return enhanced_model, final_accuracy, training_histories

# Execute

In [ ]:
final_model, accuracy, histories = train_to_perfect_verification()

print(f"\nTraining Complete!")
print(f"Final Verification Accuracy: {accuracy*100:.2f}%")

if accuracy >= 0.95:
    print("🎉 Achieved near-perfect verification accuracy!")
else:
    print(f"Current accuracy: {accuracy*100:.2f}%. Consider additional training cycles.")

Loading existing model and enhancing it...
Loading existing model from face_verification_v2.h5

✅ Model loaded successfully!
Model outputs: ['keras_tensor_43']
✅ Model recompiled successfully!
Starting advanced curriculum training...
Creating validation data for monitoring...

=== Stage 1: Enhanced Training on Diverse Data ===
Using 2000 people (limited from potentially more)
Created 5751 positive pairs
Created 5751 negative pairs
✅ Final dataset: 5751 positive, 5751 negative pairs
✅ Total pairs: 11502
Created 11502 training pairs
Starting enhanced training...
Epoch 1/25
 16/179 ━━━━━━━━━━━━━━━━━━━━ 2:36:18 58s/step - accuracy: 0.5023 - loss: 0.8441